# Traveling Salesman Problem with Google OR Tools
Solving the Traveling Salesman Problem (TSP) for the any locations on a map using Google OR Tools.

The same code can be implemented in: 
- C++
- Java
- C#

  - ### Install & import requirements
1.   OR-Tools for Python







In [34]:
!python3 -m pip install ortools

In [35]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2



- ###    Create initial data

The distance matrix is an array whose i, j entry is the distance from location i to location j in KM, where the array indices correspond to the locations in the following order:

***0-Saïda, 1-Algiers, 2-Boumerdes, 3-Oran, 4-Djelfa, 5-Béchar, 6-Ghardaïa***




In [36]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 460, 488, 175, 401, 515, 552],
        [460, 0, 46, 422, 305, 1021, 607],
        [488, 46, 0, 449, 332, 1049, 635],
        [175, 422, 446, 0, 608, 672, 666],
        [401, 305, 332, 608, 0, 663, 727],
        [515, 1021, 1049, 672, 663, 0, 790],
        [552, 607, 635, 666, 727, 790, 0],
    ] 
    data['num_vehicles'] = 1 # always = 1
    data['depot'] = 3 # The depot: the start and end location for the route.
    return data

- ### Create the routing model

The inputs to ***`RoutingIndexManager`*** are:

- The number of rows of the distance matrix, which is the number of locations (including the depot).
- The number of vehicles in the problem.
- The node corresponding to the depot.

In [37]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),
    data['num_vehicles'], 
    data['depot']
  )
  
routing = pywrapcp.RoutingModel(manager)

- ### Create the distance callback :

A function that takes any pair of locations and returns the distance between them. The easiest way to do this is using the distance matrix.


In [38]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]


The previous function creates the callback, we register it with the solver as ***`transit_callback_index`***.



In [39]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)


- ### Specify the cost of travel

The arc cost evaluator tells the solver how to calculate the cost of travel between any two locations—in other words, the cost of the edge (or arc) joining them in the graph for the problem. 

In this example, the arc cost evaluator will be the transit_callback_index, which is the solver's internal reference to the distance callback. This means that the cost of travel between any two locations is just the distance between them. However, in general the costs can involve other factors as well.


In [40]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


- ### Set search parameters

Sets the default search parameters and a heuristic method for finding the first solution... See [First Solution Strategy](https://developers.google.com/optimization/routing/routing_options#first_sol_options)




In [41]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

- ## Test and desplay solutions

In [47]:
"""Solve"""
solution = routing.SolveWithParameters(search_parameters)

"""Prints solution on console."""
index = routing.Start(0)
plan_output = 'Optimal Route :\n'
route_distance = 0
while not routing.IsEnd(index):
    plan_output += ' {} ->'.format(manager.IndexToNode(index))
    previous_index = index
    index = solution.Value(routing.NextVar(index))
    route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
plan_output += ' {}\n'.format(manager.IndexToNode(index))
print(plan_output)
print('Route distance: {}kilometers\n'.format(route_distance))

Optimal Route :
 3 -> 0 -> 6 -> 5 -> 4 -> 2 -> 1 -> 3

Route distance: 2980kilometers

